<a href="https://colab.research.google.com/github/CharlottePrimiceri/VP_Project/blob/main/depthmap/depthmap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Depth map computation**

In [ ]:
# Reference:
"""
@article{Ranftl2021,
	author    = {Ren\'{e} Ranftl and Alexey Bochkovskiy and Vladlen Koltun},
	title     = {Vision Transformers for Dense Prediction},
	journal   = {ArXiv preprint},
	year      = {2021},
}
"""

In [3]:
!pip install timm --quiet

In [4]:
import cv2
import os
import numpy as np
import torch
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

In [ ]:
model_type = "DPT_Hybrid"
midas = torch.hub.load("intel-isl/MiDaS", model_type)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
midas.to(device)
midas.eval()

midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")
if model_type == "DPT_Large" or model_type == "DPT_Hybrid":
  transform = midas_transforms.dpt_transform
else:
    transform = midas_transforms.small_transform

In [ ]:
input_folder = "/content/drive/MyDrive/VPPROJECT/cityscapes/leftImg8bit/test"
destination_folder = "/content/drive/MyDrive/VPPROJECT/cityscapes/depth_images/test"

for city in os.listdir(input_folder):
    img_dir = os.path.join(input_folder, city)
    new_dir = os.path.join(destination_folder, city)
    os.makedirs(new_dir, exist_ok=True)
    for imgid in os.listdir(img_dir):
        path = os.path.join(img_dir,imgid)
        output_path = os.path.join(destination_folder, os.path.relpath(path, input_folder))
        output_dir = os.path.dirname(output_path)
        os.makedirs(output_dir, exist_ok=True)
        img = cv2.imread(path)
        input_batch = transform(img).to(device)
        with torch.no_grad():
             prediction = midas(input_batch)
             prediction = torch.nn.functional.interpolate(
             prediction.unsqueeze(1),
             size=img.shape[:2],
             mode="bicubic",
             align_corners=False,
             ).squeeze()
        depth = prediction.cpu().numpy()
        normalized_depth = (depth - np.min(depth)) / (np.max(depth) - np.min(depth))
        depth_rgb = cv2.applyColorMap((normalized_depth * 255).astype(np.uint8), cv2.COLORMAP_JET)
        cv2.imwrite(output_path, depth_rgb)